In [ ]:
!pip install scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=fd0d879cd6cd0bbf7eb36d7372509c2c29d003fd2de776452ea29dab98bf506d
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


Input/Outputs/Membership function/Fuzzy sets

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

Fever_universe = np.arange(36, 41.1, 0.1)
Fever_points = {'Low': [36, 36.5, 37.5], 'Medium': [37, 37.5, 38.5], 'High': [38, 38.5, 41]}
Fever = create_antecedent('Fever', Fever_universe, Fever_points)

Headache_universe = np.arange(0, 10.1, 1)
Headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Intense': [6, 8, 10]}
Headache = create_antecedent('Headache', Headache_universe, Headache_points)

Cough_universe = np.arange(0, 10.1, 1)
Cough_points = {'Occasional': [0, 2, 4], 'Frequent': [3, 5, 7], 'Persistent': [6, 8, 10]}
Cough = create_antecedent('Cough', Cough_universe, Cough_points)

Diagnosis_universe = np.arange(0, 8.1, 1)
Diagnosis_points = {'Unknown': [0, 2, 4], 'Mild_viral_infection': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Flu': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Bronchitis': [5, 6, 7],
                    'Pneumonia': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Respiratory_allergy': [8, 9, 10]}
Diagnosis = create_consequent('Diagnosis', Diagnosis_universe, Diagnosis_points)


Rules for each diagnostic

In [ ]:
# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Frequent']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Occasional']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Frequent']),
              consequent=Diagnosis['Bacterial_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Respiratory_allergy'])
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)

# Create the simulation for the control system
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)


Simulation setup

In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00


In [ ]:

input_fever = widgets.FloatText(value=37.0, description='Fever (°C):', step=0.1)
input_headache = widgets.IntSlider(value=5, min=0, max=10, description='Headache:')
input_cough = widgets.IntSlider(value=5, min=0, max=10, description='Cough:')

btn_simulate = widgets.Button(description="Diagnostic Simulation")

def interpret_diagnosis(value):
    if np.isnan(value):
        return "Unknown Diagnostic"
    elif value <= 2.5:
        return "Mild viral infection"
    elif 2.5 < value <= 4.5:
        return "Common cold"
    elif 4.5 < value <= 6.5:
        return "Flu"
    elif 6.5 < value <= 8.5:
        return "Sinusitis"
    elif 8.5 < value <= 10.5:
        return "Bronchitis"
    elif 10.5 < value <= 12.5:
        return "Pneumonia"
    elif 12.5 < value <= 14.5:
        return "Bacterian infection "
    elif 14.5 < value <= 16.5:
        return "Respiratory allergy"
    else:
        return "Inconclusive diagnosis"



def on_simulate_clicked(b):
    simulation_diagnostic.input['Fever'] = input_fever.value
    simulation_diagnostic.input['Headache'] = input_headache.value
    simulation_diagnostic.input['Cough'] = input_cough.value

    try:
        simulation_diagnostic.compute()
        diagnosis_text = interpret_diagnosis(simulation_diagnostic.output['Diagnosis'])
    except ValueError:
        diagnosis_text = "Inconclusive diagnosis"

    clear_output(wait=True)
    display(input_fever, input_headache, input_cough, btn_simulate)
    print(f"Suggested diagnosis: {diagnosis_text}")

btn_simulate.on_click(on_simulate_clicked)


display(input_fever, input_headache, input_cough, btn_simulate)


FloatText(value=37.0, description='Fever (°C):', step=0.1)

IntSlider(value=5, description='Headache:', max=10)

IntSlider(value=5, description='Cough:', max=10)

Button(description='Diagnostic Simulation', style=ButtonStyle())

In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Define symptom and diagnosis options
symptom_options = ['Fever', 'Headache', 'Cough']
diagnosis_options = ['Unknown', 'Mild viral infection', 'Common cold', 'Flu',
                     'Sinusitis', 'Bronchitis', 'Pneumonia', 'Bacterial infection',
                     'Respiratory allergy']

# Define interactive widgets with styling
symptom_dropdown = widgets.Dropdown(options=symptom_options, description='Symptom:',
                                     style={'description_width': 'initial', 'width': '300px'})
symptom_value = widgets.FloatSlider(value=37.0, min=36.0, max=41.0, step=0.1, description='Value:',
                                    style={'description_width': 'initial', 'width': '300px'})
diagnosis_dropdown = widgets.Dropdown(options=diagnosis_options, description='Diagnosis:',
                                      style={'description_width': 'initial', 'width': '300px'})
btn_add_symptom = widgets.Button(description="Add Symptom", button_style='success',
                                  layout={'width': '150px'})
btn_remove_symptom = widgets.Button(description="Remove Symptom", button_style='danger',
                                     layout={'width': '150px'})
btn_diagnose = widgets.Button(description="Diagnose", button_style='info', layout={'width': '150px'})
output_diagnosis = widgets.Output()

# Define list to store selected symptoms
selected_symptoms = []

# Define function to handle adding symptom
def add_symptom(b):
    selected_symptoms.append((symptom_dropdown.value, symptom_value.value))
    with output_diagnosis:
        clear_output(wait=True)
        print("Symptom added:", selected_symptoms)

# Define function to handle removing symptom
def remove_symptom(b):
    if selected_symptoms:
        selected_symptoms.pop()
        with output_diagnosis:
            clear_output(wait=True)
            print("Symptom removed:", selected_symptoms)

# Define function to handle diagnosis
def diagnose(b):
    if selected_symptoms:
        clear_output(wait=True)
        display(symptom_dropdown, symptom_value, btn_add_symptom, btn_remove_symptom, btn_diagnose)
        print("Selected Symptoms:")
        for symptom, value in selected_symptoms:
            print(f"- {symptom}: {value}")
        print("\nDiagnosis:")
        # Perform diagnostic inference here based on selected symptoms
        # Display diagnostic results using output_diagnosis widget

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)

# Display initial widgets with colorful styling
display(widgets.VBox([
    widgets.HBox([symptom_dropdown, symptom_value]),
    widgets.HBox([btn_add_symptom, btn_remove_symptom, btn_diagnose]),
    output_diagnosis
]), layout={'align_items': 'center', 'justify_content': 'center', 'width': '80%', 'margin': '0 auto'})


In [ ]:
def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

Fever_universe = np.arange(36, 41.1, 0.1)
Fever_points = {'Low': [36, 36.5, 37.5], 'Medium': [37, 37.5, 38.5], 'High': [38, 38.5, 41]}
Fever = create_antecedent('Fever', Fever_universe, Fever_points)

Headache_universe = np.arange(0, 10.1, 1)
Headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Intense': [6, 8, 10]}
Headache = create_antecedent('Headache', Headache_universe, Headache_points)

Cough_universe = np.arange(0, 10.1, 1)
Cough_points = {'Occasional': [0, 2, 4], 'Frequent': [3, 5, 7], 'Persistent': [6, 8, 10]}
Cough = create_antecedent('Cough', Cough_universe, Cough_points)

Diagnosis_universe = np.arange(0, 8.1, 1)
Diagnosis_points = {'Unknown': [0, 2, 4], 'Mild_viral_infection': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Flu': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Bronchitis': [5, 6, 7],
                    'Pneumonia': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Respiratory_allergy': [8, 9, 10]}
Diagnosis = create_consequent('Diagnosis', Diagnosis_universe, Diagnosis_points)

# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Frequent']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Occasional']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Frequent']),
              consequent=Diagnosis['Bacterial_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Respiratory_allergy'])
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)

# Create the simulation for the control system
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)


# Define symptom and diagnosis options
symptom_options = ['Fever', 'Headache', 'Cough']
diagnosis_options = ['Unknown', 'Mild viral infection', 'Common cold', 'Flu',
                     'Sinusitis', 'Bronchitis', 'Pneumonia', 'Bacterial infection',
                     'Respiratory allergy']

# Define interactive widgets with styling
symptom_dropdown = widgets.Dropdown(options=symptom_options, description='Symptom:',
                                     style={'description_width': 'initial', 'width': '300px'})
symptom_value = widgets.FloatSlider(value=37.0, min=36.0, max=41.0, step=0.1, description='Value:',
                                    style={'description_width': 'initial', 'width': '300px'})
diagnosis_dropdown = widgets.Dropdown(options=diagnosis_options, description='Diagnosis:',
                                      style={'description_width': 'initial', 'width': '300px'})
btn_add_symptom = widgets.Button(description="Add Symptom", button_style='success',
                                  layout={'width': '150px'})
btn_remove_symptom = widgets.Button(description="Remove Symptom", button_style='danger',
                                     layout={'width': '150px'})
btn_diagnose = widgets.Button(description="Diagnose", button_style='info', layout={'width': '150px'})
output_diagnosis = widgets.Output()

# Define list to store selected symptoms
selected_symptoms = []

# Define function to handle adding symptom
def add_symptom(b):
    selected_symptoms.append((symptom_dropdown.value, symptom_value.value))
    with output_diagnosis:
        clear_output(wait=True)
        print("Symptom added:", selected_symptoms)

# Define function to handle removing symptom
def remove_symptom(b):
    if selected_symptoms:
        selected_symptoms.pop()
        with output_diagnosis:
            clear_output(wait=True)
            print("Symptom removed:", selected_symptoms)

# Define function to handle diagnosis
def diagnose(b):
    if selected_symptoms:
        clear_output(wait=True)
        display(symptom_dropdown, symptom_value, btn_add_symptom, btn_remove_symptom, btn_diagnose)
        print("Selected Symptoms:")
        for symptom, value in selected_symptoms:
            print(f"- {symptom}: {value}")
        print("\nDiagnosis:")
        # Perform diagnostic inference here based on selected symptoms
        # Display diagnostic results using output_diagnosis widget

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)

# Display initial widgets with colorful styling
display(widgets.VBox([
    widgets.HBox([symptom_dropdown, symptom_value]),
    widgets.HBox([btn_add_symptom, btn_remove_symptom, btn_diagnose]),
    output_diagnosis
]), layout={'align_items': 'center', 'justify_content': 'center', 'width': '80%', 'margin': '0 auto'})



///////////////////////////////////////////////////////////////////////////


New improved version( still needs commentaries and check for improvements )

In [ ]:
# Install necessary packages
!pip install ipywidgets
!pip install scikit-fuzzy

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [4]:


# Define functions to create membership functions, antecedents, and consequents
def define_membership_function(universe, points):
    return fuzz.trimf(universe, points)

def create_antecedent(name, universe, points_dict):
    antecedent = ctrl.Antecedent(universe, name)
    for label, points in points_dict.items():
        antecedent[label] = define_membership_function(universe, points)
    return antecedent

def create_consequent(name, universe, points_dict):
    consequent = ctrl.Consequent(universe, name)
    for label, points in points_dict.items():
        consequent[label] = define_membership_function(universe, points)
    return consequent

# Define universes and membership points for each symptom and diagnosis
Fever_universe = np.arange(36, 41.1, 0.1)
Fever_points = {'Low': [36, 36.5, 37.5], 'Medium': [37, 37.5, 38.5], 'High': [38, 38.5, 41]}
Fever = create_antecedent('Fever', Fever_universe, Fever_points)

Headache_universe = np.arange(0, 10.1, 1)
Headache_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Intense': [6, 8, 10]}
Headache = create_antecedent('Headache', Headache_universe, Headache_points)

Cough_universe = np.arange(0, 10.1, 1)
Cough_points = {'Occasional': [0, 2, 4], 'Frequent': [3, 5, 7], 'Persistent': [6, 8, 10]}
Cough = create_antecedent('Cough', Cough_universe, Cough_points)

Fatigue_universe = np.arange(0, 10.1, 1)
Fatigue_points = {'Low': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
Fatigue = create_antecedent('Fatigue', Fatigue_universe, Fatigue_points)

SoreThroat_universe = np.arange(0, 10.1, 1)
SoreThroat_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
SoreThroat = create_antecedent('SoreThroat', SoreThroat_universe, SoreThroat_points)

ShortnessOfBreath_universe = np.arange(0, 10.1, 1)
ShortnessOfBreath_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
ShortnessOfBreath = create_antecedent('ShortnessOfBreath', ShortnessOfBreath_universe, ShortnessOfBreath_points)

Nausea_universe = np.arange(0, 10.1, 1)
Nausea_points = {'Mild': [0, 2, 4], 'Moderate': [3, 5, 7], 'Severe': [6, 8, 10]}
Nausea = create_antecedent('Nausea', Nausea_universe, Nausea_points)

Diagnosis_universe = np.arange(0, 10.1, 1)
Diagnosis_points = {'Unknown': [0, 1, 2], 'Mild_viral_infection': [1, 2, 3], 'Common_cold': [2, 3, 4],
                    'Flu': [3, 4, 5], 'Sinusitis': [4, 5, 6], 'Bronchitis': [5, 6, 7],
                    'Pneumonia': [6, 7, 8], 'Bacterial_infection': [7, 8, 9], 'Respiratory_allergy': [8, 9, 10]}
Diagnosis = create_consequent('Diagnosis', Diagnosis_universe, Diagnosis_points)

# Define rules mapping input patterns to diagnoses
rules = [
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Frequent']),
              consequent=Diagnosis['Mild_viral_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Occasional']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Common_cold']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Flu']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Intense'] & Cough['Occasional']),
              consequent=Diagnosis['Sinusitis']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Low'] & Cough['Persistent']),
              consequent=Diagnosis['Bronchitis']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Moderate'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Persistent']),
              consequent=Diagnosis['Pneumonia']),
    ctrl.Rule(antecedent=(Fever['High'] & Headache['Intense'] & Cough['Frequent']),
              consequent=Diagnosis['Bacterial_infection']),
    ctrl.Rule(antecedent=(Fever['Low'] & Headache['Moderate'] & Cough['Frequent']),
              consequent=Diagnosis['Respiratory_allergy']),
    ctrl.Rule(antecedent=(Fever['Medium'] & Headache['Low'] & Cough['Occasional']),
              consequent=Diagnosis['Respiratory_allergy'])
]

# Create the control system with the defined rules
diagnostic_system = ctrl.ControlSystem(rules)

# Create the simulation for the control system
simulation_diagnostic = ctrl.ControlSystemSimulation(diagnostic_system)

# Define the UI components
symptom_dropdown = widgets.Dropdown(
    options=['Fever', 'Headache', 'Cough', 'Fatigue', 'Sore Throat', 'Shortness of Breath', 'Nausea'],
    description='Symptom:',
    style={'description_width': 'initial'}
)
symptom_value = widgets.FloatSlider(
    value=37.0, min=36.0, max=41.0, step=0.1,
    description='Value:',
    style={'description_width': 'initial'}
)
btn_add_symptom = widgets.Button(description="Add Symptom", button_style='success')
btn_remove_symptom = widgets.Button(description="Remove Symptom", button_style='danger')
btn_diagnose = widgets.Button(description="Diagnose", button_style='info')
output_diagnosis = widgets.Output()

# Define list to store selected symptoms
selected_symptoms = []

# Define function to update the symptom value range based on the selected symptom
def update_symptom_value_range(change):
    symptom = change['new']
    if symptom == 'Fever':
        symptom_value.min = 36.0
        symptom_value.max = 41.0
        symptom_value.step = 0.1
    else:
        symptom_value.min = 0.0
        symptom_value.max = 10.0
        symptom_value.step = 0.1
    symptom_value.value = symptom_value.min

symptom_dropdown.observe(update_symptom_value_range, names='value')

# Define function to handle adding symptom
def add_symptom(b):
    selected_symptoms.append((symptom_dropdown.value, symptom_value.value))
    with output_diagnosis:
        clear_output(wait=True)
        print("Symptom added:", selected_symptoms)

# Define function to handle removing symptom
def remove_symptom(b):
    if selected_symptoms:
        selected_symptoms.pop()
        with output_diagnosis:
            clear_output(wait=True)
            print("Symptom removed:", selected_symptoms)

# Define function to handle diagnosis
def diagnose(b):
    if selected_symptoms:
        with output_diagnosis:
            clear_output(wait=True)
            print("Selected Symptoms:")
            for symptom, value in selected_symptoms:
                print(f"- {symptom}: {value}")
            print("\nDiagnosis:")
            for symptom, value in selected_symptoms:
                key = symptom.lower().replace(" ", "")
                simulation_diagnostic.input[key] = value
            simulation_diagnostic.compute()
            diagnosis_result = simulation_diagnostic.output['Diagnosis']
            print("Diagnosis result:", diagnosis_result)

# Bind event handlers to buttons
btn_add_symptom.on_click(add_symptom)
btn_remove_symptom.on_click(remove_symptom)
btn_diagnose.on_click(diagnose)

# Display initial widgets
display(widgets.VBox([
    widgets.HBox([symptom_dropdown, symptom_value]),
    widgets.HBox([btn_add_symptom, btn_remove_symptom, btn_diagnose]),
    output_diagnosis
]))
